### 1. Preparing data for analyses

In [6]:
import arcpy
import os

# set path
mypath = r"C:\Users\hanna\OneDrive\Documents\ArcGIS\Projects\QOZ_ResPerm"

In [7]:
# variables for geodatabase
gdb = "QOZ_ResPerm.gdb"

In [8]:
# set environment and allow overwrite
arcpy.env.workspace = os.path.join(mypath, gdb)
arcpy.env.overwriteOutput = True

Here I want to convert the shapefiles to feature classes and then import them into my geodatabase (*QOZ_ResPerm.gdb*).

In [ ]:
# import shapefiles/feature classes into the GDB
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features=[
        r"QOZ",
        r"Residential_Construction_Permits_by_County"
    ],
    Output_Geodatabase=os.path.join(mypath, gdb)
)

In [9]:
# list the feature classes gdb
fc_list = arcpy.ListFeatureClasses()
print(fc_list)

['QOZ', 'Residential_Construction_Permits_by_County']


### 2. Intersection

Now, I want to run an intersection of see how many building permits fall within QOZs.

In [13]:
# create variables for the feature classes
# this will improve readability
counties = "Residential_Construction_Permits_by_County"
qoz      = "QOZ"

In [14]:
# output of the intersection
output_fc = "permit_qoz_intersect"

In [15]:
# run the intersection using created variables
arcpy.analysis.Intersect(
    in_features=[counties, qoz],  # the permits and QOZs are the input
    out_feature_class=output_fc,  # output 
    join_attributes="ALL",         
    cluster_tolerance=None, 
    output_type="INPUT"            
)
print(f"Created: {output_fc}")

Created: permit_qoz_intersect


This intersection communicates that QOZs may not have much impact on permit acquisition.

One also needs to take into account that many of the QOZs with considerable residential permits are in regions/cities that have a high population. So, it would make sense that there is higher demand for permits in those regions. 

### 3. Buffer

Now I'm using a 5 mile buffer to see how big the scope of QOZs are. 

In [ ]:
# 5 mile buffer around QOZs 
arcpy.analysis.Buffer(
    in_features="QOZ",
    out_feature_class=r"QOZ_Buffer",
    buffer_distance_or_field="5 Miles",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="NONE",
    dissolve_field=None,
    method="PLANAR"
)

The point of visualizing buffers around the QOZs is to assess if the tax incentivized land are in areas that seem viable for increased buying of property. I want to clearly visualize if QOZs are close to areas where there is high building acquisition to show the expected effectiveness of the tax incentives. The logic being if QOZs are closer to areas with higher building permit acquisition then the more likely the QOZs will be taken advantage of, indicating an effectiveness in creating the opportunity zones. 

### 4. Select tool to isolate desired states

Using the select tool, I create layers that only have my states of interest (FL, TX, CA)

In [ ]:
# selecting QOZ and building permit data for california, texas, florida, puerto rice
# select building permit
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="Residential_Construction_Permits_by_County",
    selection_type="NEW_SELECTION",
    where_clause="STATE_NAME = 'California'", #'Florida'
    invert_where_clause="NON_INVERT"
)

# select by QOZ buffers
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="QOZ_Buffer",
    selection_type="NEW_SELECTION",
    where_clause="STATE_NAME = 'California'", #'Florida'
    invert_where_clause="NON_INVERT"
)

# select by QOZS
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="QOZ",
    selection_type="NEW_SELECTION",
    where_clause="STATE_NAME = 'California'", #'Florida''
    invert_where_clause="NON_INVERT"
)

Now I'm interested in seeing how QOZs without buffers to get a general idea of where these tax incentivized areas lie within the areas with high building permits. 

## Using the dissolve tool to make QOZ "boundaries"

I'm using the dissolve tool here to get a sense of what areas within the states would be the "most eligible" for further development. From that I can surmise which areas within/around QOZs are most likely to be taken advantage of based on the demand (or amount) of building permits within a given county.

In [1]:
# dissolve FL QOZ buffers into QOZ likely development areas
arcpy.gapro.DissolveBoundaries(
    input_layer="Florida QOZ",
    out_feature_class=r"Florida_Dev_Area",
    multipart="SINGLE_PART",
    dissolve_fields="DISSOLVE_FIELDS",
    fields="BUFF_DIST",
    summary_fields=None
)

arcpy.gapro.DissolveBoundaries(
    input_layer="California QOZ",
    out_feature_class=r"California_Dev_Area",
    multipart="SINGLE_PART",
    dissolve_fields="DISSOLVE_FIELDS",
    fields="BUFF_DIST",
    summary_fields=None
)

<Result 'C:\\Users\\hanna\\OneDrive\\Documents\\ArcGIS\\Projects\\QOZ_ResPerm\\QOZ_ResPerm.gdb\\California_Dev_Area'>

# Analysis of Results

This spatial analysis reveals critical insights for community development, investment strategy, and policy evaluation at the intersection of Qualified Opportunity Zones and construction activity. High-permit counties show notable overlap with QOZ-adjacent areas, suggesting that federal tax incentives are successfully catalyzing development in economically distressed communities. This pattern is seen within Southern California and Florida's urban areas. However, this pattern raises important questions about equitable growth and the risk of gentrification.

For investors, target regions where green outlines overlap with the deepest purple shading to maximize QOZ tax benefits and access markets demonstrating construction activity. Policymakers should assess if development leads to affordable housing and community amenities. Without proper intention toward the residents within these areas, it may invite projects that accelerate community displacement.
Communities in QOZ areas with high permit acquisition rates face both opportunities and risks. New investment can bring jobs, infrastructure, and services. However, without intentional planning and community benefit agreements, these forces may also give rise to displacement.

According to the IRS QOZ frequently asked questions, investors do not need to be part of the community before taking advantage of these tax incentives. If looking to support distressed economies, the focus should be on housing affordability, demographic shifts, and local hiring outcomes in these zones before implementing tax incentives. This will help assess whether QOZ incentives achieve their intended purpose of inclusive economic revitalization.
